In [1]:
import tensorflow as tf
import numpy as np
import mnist_data
import os
# import beta_vae as vae
import vae
import plot_utils
import glob
import random
import matplotlib.pyplot as plt
import cv2
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [2]:
#gpu
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
#不平衡 类别
choose_c=3
#图像尺寸
IMAGE_SIZE_MNIST = 28
#隐变量维度
dim_z=3
#alpha classification loss balance
alpha=400
#隐层节点数
n_hidden=500
#学习率
learn_rate=1e-3
#训练轮数
n_epochs=100
#批量数目
batch_size=256
#标签样式 one-hot编码
one_hot=np.eye(10)
np.random.seed(100)
dim_img=28*28

In [3]:
_, _, _, _, test_data, test_labels = mnist_data.prepare_MNIST_data()

W0918 10:20:56.365740 140399666411328 deprecation_wrapper.py:119] From /home/yuemei.zhu/tensorflow-mnist-VAE/mnist_data.py:30: The name tf.gfile.Exists is deprecated. Please use tf.io.gfile.exists instead.



Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


In [4]:
x_hat = tf.placeholder(tf.float32, shape=[None, dim_img], name='input_img')
x_label = tf.placeholder(tf.float32, shape=[None, 10], name='img_label')
keep_prob = tf.placeholder(tf.float32, name='keep_prob')

In [5]:
train_rebanlance_dis_label=np.load('./train_rebanlance_dis_label.npy')
np.random.shuffle(train_rebanlance_dis_label)

### 分类器结构

In [6]:
"""CNN 的模型函数"""
# 输入层
input_layer = tf.reshape(x_hat, [-1, 28, 28, 1])
# 第一个卷积层
conv1 = tf.layers.conv2d(
  inputs=input_layer,
  filters=32,
  kernel_size=[5, 5],
  padding="same",
  activation=tf.nn.relu)

# 第一个池化层
pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

# 第二个卷积层和池化层
conv2 = tf.layers.conv2d(
  inputs=pool1,
  filters=64,
  kernel_size=[5, 5],
  padding="same",
  activation=tf.nn.relu)
pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

# 全连接层
pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
dropout=tf.nn.dropout(dense, keep_prob)
# dropout = tf.layers.dropout(
#   inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

# Logits 层
logits = tf.layers.dense(inputs=dropout, units=10)
# predicts=tf.argmax(logits,axis=1)
# 计算损失（可用于`训练`和`评价`中）
cls_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=x_label, logits=logits))
cls_optm = tf.train.GradientDescentOptimizer(learning_rate=0.001)
cls_train_op = cls_optm.minimize(cls_loss)

W0918 10:20:58.097620 140399666411328 deprecation.py:323] From <ipython-input-6-d38760c62036>:10: conv2d (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
W0918 10:20:58.103189 140399666411328 deprecation.py:506] From /home/yuemei.zhu/.local/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0918 10:20:58.336005 140399666411328 deprecation.py:323] From <ipython-input-6-d38760c62036>:13: max_pooling2d (from tensorflow.python.layers.pooling) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
W0918 10:20:58.496815 140399666411328 deprecat

### 在类别重新均衡的数据集上训练分类器

In [7]:
cls_epochs=100
cls_batch_size=128
batchs=int(train_rebanlance_dis_label.shape[0]/cls_batch_size)
sess=tf.Session()
sess.run(tf.global_variables_initializer(), feed_dict={keep_prob : 0.9})
for epoch in range(cls_epochs):
    loss_np=0
    for i in range(batchs):
        batch_xs_input=train_rebanlance_dis_label[i*cls_batch_size:(i+1)*cls_batch_size,:-10]
        batch_xs_label=train_rebanlance_dis_label[i*cls_batch_size:(i+1)*cls_batch_size,-10:]
        _, cls_loss_np= sess.run(
            (cls_train_op, cls_loss),
            feed_dict={x_hat: batch_xs_input,x_label:batch_xs_label,keep_prob : 0.9})
        loss_np+=cls_loss_np
    print('epoch %d : % d epochs loss %3.2f '% (epoch,cls_epochs,loss_np/batchs))

epoch 0 :  100 epochs loss 2.27 
epoch 1 :  100 epochs loss 2.16 
epoch 2 :  100 epochs loss 1.84 
epoch 3 :  100 epochs loss 1.14 
epoch 4 :  100 epochs loss 0.66 
epoch 5 :  100 epochs loss 0.48 
epoch 6 :  100 epochs loss 0.40 
epoch 7 :  100 epochs loss 0.35 
epoch 8 :  100 epochs loss 0.32 
epoch 9 :  100 epochs loss 0.29 
epoch 10 :  100 epochs loss 0.27 
epoch 11 :  100 epochs loss 0.26 
epoch 12 :  100 epochs loss 0.24 
epoch 13 :  100 epochs loss 0.23 
epoch 14 :  100 epochs loss 0.22 
epoch 15 :  100 epochs loss 0.21 
epoch 16 :  100 epochs loss 0.21 
epoch 17 :  100 epochs loss 0.20 
epoch 18 :  100 epochs loss 0.19 
epoch 19 :  100 epochs loss 0.18 
epoch 20 :  100 epochs loss 0.18 
epoch 21 :  100 epochs loss 0.17 
epoch 22 :  100 epochs loss 0.17 
epoch 23 :  100 epochs loss 0.16 
epoch 24 :  100 epochs loss 0.16 
epoch 25 :  100 epochs loss 0.16 
epoch 26 :  100 epochs loss 0.15 
epoch 27 :  100 epochs loss 0.15 
epoch 28 :  100 epochs loss 0.14 
epoch 29 :  100 epochs l

In [8]:
cls_batch_size=2
batchs=int(test_data.shape[0]/cls_batch_size)
den=np.sum(test_labels,axis=0)
num=np.zeros([1,10])
for epoch in range(1):
#     loss_np=0
    corrects=0
    for i in range(batchs):
        batch_xs_input=test_data[i*cls_batch_size:(i+1)*cls_batch_size]
        batch_xs_label=test_labels[i*cls_batch_size:(i+1)*cls_batch_size]
        predicts= sess.run(logits,
                                 feed_dict={x_hat: batch_xs_input,x_label:batch_xs_label,keep_prob : 1})
        num+=np.sum(one_hot[np.argmax(batch_xs_label,axis=1)]*one_hot[np.argmax(predicts,axis=1)],axis=0)
        corrects+=np.sum(np.argmax(predicts,axis=1)==np.argmax(batch_xs_label,axis=1))
        #         loss_np+=cls_loss_np
    print('acc %3.2f '% (corrects/batchs/cls_batch_size))
    print('recall : ',num/den)

acc 0.97 
recall :  [[0.99285714 0.9876652  0.97965116 0.88613861 0.98065173 0.98318386
  0.9822547  0.97178988 0.97843943 0.95936571]]
